<a href="https://colab.research.google.com/github/Mgrinet/classificador-CNN-vgg16/blob/master/CatDog_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensoes das imagens.
img_width, img_height = 150, 150

train_data_dir = '/data/train'
validation_data_dir = '/data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 10
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Configuracao da augumentacao e imagem que vamos usar para treinamento
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Configuracao da augumentacao e imagem que vamos usar para teste (apenas rescaling)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('/vgg16_weights.h5')

Using TensorFlow backend.
W0831 12:08:50.901671 140108253095808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 12:08:50.944645 140108253095808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 12:08:50.951870 140108253095808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0831 12:08:50.986112 140108253095808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0831 12:08:51.050211 1401082

Found 3000 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Epoch 1/10
125/125 [==============================] - 1011s 8s/step - loss: 0.6589 - acc: 0.6630 - val_loss: 1.3626 - val_acc: 0.0000e+00
Epoch 2/10
125/125 [==============================] - 394s 3s/step - loss: 0.6358 - acc: 0.6660 - val_loss: 0.8214 - val_acc: 0.0286
Epoch 3/10
125/125 [==============================] - 21s 170ms/step - loss: 0.6102 - acc: 0.6700 - val_loss: 1.2734 - val_acc: 0.0494
Epoch 4/10
125/125 [==============================] - 24s 189ms/step - loss: 0.5816 - acc: 0.6715 - val_loss: 1.1484 - val_acc: 0.2195
Epoch 5/10
125/125 [==============================] - 22s 173ms/step - loss: 0.5539 - acc: 0.7210 - val_loss: 2.3187 - val_acc: 0.0169
Epoch 6/10
125/125 [==============================] - 21s 167ms/step - loss: 0.5632 - acc: 0.7160 - val_loss: 0.9609 - val_acc: 0.3364
Epoch 7/10
125/125 [==============================] - 23s 184ms/step - loss: 0.5411 - acc: 0.7120 - val_lo

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense


weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'fc_model.h5'

img_width, img_height = 150, 150

train_data_dir = '/data/train'
validation_data_dir = '/data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# construir rede VGG16 
model = applications.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# modelo pra colocar em cima do conv treinado
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# precisa ter um modelo ja treinado (classificador)
top_model.load_weights(top_model_weights_path)

# adicionar modelo em cima da base convolucional
model.add(top_model)

# primeiras 25 camadas nao serao treinadas 
for layer in model.layers[:25]:
    layer.trainable = False

# compilar modelo
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# augumentacao
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune o modelo
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)